In [ ]:
using Random
using BenchmarkTools

mutable struct MyObj
    index::Int32
    symbol::Symbol
    value::Float64
end

objs = [MyObj(i, Symbol("a"*string(i)), rand()) for i in 1:1_000_000]
objs

In [14]:
vec_index = objs
dict_index = Dict(obj.symbol => obj for (i, obj) in enumerate(objs))

function test_vec(vec::Vector{MyObj}, objs::Vector{MyObj})
    sum = 0.0
    for obj in objs
        sum += @inbounds vec[obj.index].value
    end
    sum
end

function test_dict(vec::Dict{Symbol,MyObj}, objs::Vector{MyObj})
    sum = 0.0
    for obj in objs
        sum += @inbounds vec[obj.symbol].value
    end
    sum
end

objs_shuffled = shuffle(objs)
display(@benchmark test_vec(vec_index, objs_shuffled) samples=20 evals=2)
display(@benchmark test_dict(dict_index, objs_shuffled) samples=20 evals=2)

BenchmarkTools.Trial: 20 samples with 2 evaluations.
 Range (min … max):  8.822 ms …  10.591 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.753 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.641 ms ± 419.786 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▃                      █                           
  ▇▁▁▁▁▁▁▁▇▁█▁▁▁▁▇▁▇▁▇▁▁▁▁▇▁▁▁▇▁▇▇▁█▇▇▁▇▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  8.82 ms         Histogram: frequency by time        10.6 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

BenchmarkTools.Trial: 20 samples with 2 evaluations.
 Range (min … max):  49.592 ms …  50.697 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     49.915 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.006 ms ± 376.566 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▁█ █▁ ▁ ▁      ▁  ▁  ▁     █            ▁▁   ▁   ▁     ▁  ▁  
  ███▁██▁█▁█▁▁▁▁▁▁█▁▁█▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁█▁▁▁█▁▁▁▁▁█▁▁█ ▁
  49.6 ms         Histogram: frequency by time         50.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [ ]:
@code_warntype test_vec(vec_index, objs_shuffled)

In [ ]:
@code_warntype test_dict(dict_index, objs_shuffled)

In [13]:
using ProfileView
ProfileView.@profview (for i = 1:1000; test_vec(vec_index, objs_shuffled); end)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=2, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  21:33:39", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip